# First object condensation model training

* **Requirements*: You need to have graphs constructed, e.g., with `010_build_graphs.ipynb`

In [2]:
from pathlib import Path

from gnn_tracking.postprocessing.dbscanscanner import dbscan_scan

from gnn_tracking.models.track_condensation_networks import GraphTCN
from gnn_tracking.training.tcn_trainer import TCNTrainer
from gnn_tracking.metrics.losses import (
    EdgeWeightFocalLoss,
    PotentialLoss,
    BackgroundLoss,
)
from gnn_tracking.utils.loading import get_loaders, TrackingDataset

In [3]:
data_path = Path.home() / "ws/CERN/gnn-tracking/data" 
graph_dir = data_path / "graphs"
print(graph_dir)
assert graph_dir.is_dir()

/homes/zque/ws/CERN/gnn-tracking/data/graphs


In [4]:
datasets = {
    "train": TrackingDataset(graph_dir, stop=9),
    "val": TrackingDataset(graph_dir, start=9, stop=10),
}
loaders = get_loaders(datasets)

[23:44:41] INFO: DataLoader will load 9 graphs (out of 10 available).
[23:44:41] DEBUG: First graph is /homes/zque/ws/CERN/gnn-tracking/data/graphs/data2800_s0.pt, last graph is /homes/zque/ws/CERN/gnn-tracking/data/graphs/data2800_s16.pt
[23:44:41] INFO: DataLoader will load 1 graphs (out of 10 available).
[23:44:41] DEBUG: First graph is /homes/zque/ws/CERN/gnn-tracking/data/graphs/data2800_s17.pt, last graph is /homes/zque/ws/CERN/gnn-tracking/data/graphs/data2800_s17.pt
[23:44:41] DEBUG: Parameters for data loader 'train': {'batch_size': 1, 'num_workers': 1, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x7f60f438cc10>, 'pin_memory': True, 'shuffle': None}
[23:44:41] DEBUG: Parameters for data loader 'val': {'batch_size': 1, 'num_workers': 1, 'sampler': None, 'pin_memory': True, 'shuffle': False}


In [5]:
loss_functions = {
    "edge": (EdgeWeightFocalLoss(gamma=5, alpha=0.95), 500.0),
    "potential": (PotentialLoss(q_min=0.01), {"attractive": 500, "repulsive": 5}),
    "background": (BackgroundLoss(sb=1), 0.05),
}

The values after the loss functions are the loss weights. The potential loss is a special case, because it returns a dictionary two values: `attractive` and `repulsive`. Therefore, there are also two loss weights.

In [6]:
model = GraphTCN(
    node_indim=datasets["train"].num_node_features,
    edge_indim=datasets["train"].num_edge_features,
    h_dim=10,
    e_dim=10,
    L_ec=5,
    L_hc=2,
    h_outdim=10,
    hidden_dim=128,
)

In [7]:
clustering_functions = {"dbscan": dbscan_scan}

In [8]:
trainer = TCNTrainer(
    model=model,
    loaders=loaders,
    loss_functions=loss_functions,
    lr=0.005,
    cluster_functions=clustering_functions,
)

[23:45:01 TCNTrainer] INFO: Using device cuda


In [9]:
trainer.train(epochs=2)

/homes/zque/ws/anaconda3/envs/gnn-tracking/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "<string>", line 3, in _binary_focal_loss

def mul(a : float, b : Tensor) -> Tensor:
  return b * a
         ~~~~~ <--- HERE
def add(a : float, b : Tensor) -> Tensor:
  return b + a
RuntimeError: Expected a proper Tensor but got None (or an undefined Tensor in C++) for argument #0 'self'
